# Panda3D Tutorial 2

![](https://www.panda3d.org/wp-content/uploads/2018/12/panda3d_logo_s_white.png)

_Mikołaj Leszczuk_

Na podstawie: [Panda3D Beginner's Tutorial](https://arsthaumaturgis.github.io/Panda3DTutorial.io/)

## Dźwięk

Nauczmy się obsługi dźwięku i muzyki w Panda3D.

W większości przypadków radzenie sobie z dźwiękiem i muzyką w Panda3D jest dość proste. Wczytujecie plik dźwiękowy lub muzyczny, odtwarzacie go (być może po ustawieniu głośności lub zapętleniu) i to wszystko!

Można zrobić bardziej złożone rzeczy (takie jak użycie dźwięku pozycyjnego 3D), ale do naszych celów wystarczy powyższe.

Warto zauważyć, że Panda rozróżnia efekty dźwiękowe i muzykę; ma dla nich osobne obiekty menedżerskie. Ułatwia to pracę z jednym z nich jako całością bez wpływu na drugie. Na przykład możecie dostosować ogólną głośność muzyki, pozostawiając efekty dźwiękowe bez zmian.

Na początek załadujmy i odtwórzmy muzykę. W większym projekcie możemy mieć wiele plików muzycznych, które będą odtwarzane w różnych okolicznościach, ale do celów nauki będziemy mieli tylko jeden utwór muzyczny ("Vitamins"/[Sundial Aeon](http://sundial-aeon.pl)).

```python
# W metodzie „__init__”:
music = loader.loadMusic("Vitamins.ogg")
music.setLoop(True)
# Ten utwór jest dość głośny,
# więc ściszmy głośność.
# Dostosujcie się do swoich ustawień i preferencji!
music.setVolume(0.75)
music.play()
```

```python
from direct.showbase.ShowBase import ShowBase


class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # W metodzie „__init__”:
        music = self.loader.loadMusic("Vitamins.ogg")
        music.setLoop(True)
        # Ten utwór jest dość głośny,
        # więc ściszmy głośność.
        # Dostosujcie się do swoich ustawień i preferencji!
        music.setVolume(0.75)
        music.play()


app = MyApp()
app.run()
```

Powinniście teraz usłyszeć muzykę!

Być może pamiętacie, że aby zapętlić animację w aktorze, po prostu wywołujemy „pętlę”. Zauważcie zatem, że nie dotyczy to dźwięków: zamiast tego najpierw wywołujemy „setLoop”, który nie uruchamia dźwięku, ale wskazuje, że powinien zapętlić się podczas odtwarzania, a następnie wywołujemy „play”, który rozpoczyna to.

## Pakowanie do postaci pliku uruchamialnego

Dowiedzmy się, jak zbudować dystrybucyjną wersję naszej aplikacji

Panda3D zawiera narzędzie, które – z pewnymi wskazówkami od nas – automatycznie zbuduje dystrybucyjną wersję naszej aplikacji, pobierając pliki z Internetu zgodnie z wymaganiami.

Wymaga dwóch plików:
* „requirements.txt”, który mówi, jakie pakiety zależności będą wymagane przez ostateczną aplikację. (I zauważcie, że sam pakiet „panda3d” jest taką zależnością!)
* I plik nazwany dowolnie, powiedzmy: „setup.py”, który szczegółowo opisuje, jak powinien zbudować aplikację.

Plik „requirements.txt” jest dość prosty. Rzeczywiście, dla naszych celów będzie miał tylko jedną linię:

```
panda3d
```

Plik „setup.py” jest nieco bardziej skomplikowany.

Na początek podstawy: z „setuptools” importujemy metodę „setup”, a następnie ją wywołujemy, podając szczegóły naszej aplikacji jako parametry. Szablon tego kodu wygląda tak:

```python
from setuptools import setup

setup(
# Tutaj idą parametry...
)
```

Dajmy mu to dwa parametry: nazwa naszej aplikacji oraz słownik opcji.

W naszym przypadku technicznie będziemy mieli tylko jedną opcję - ale ta "opcja" sama będzie zawierać kilka opcji. Ta opcja nazywa się „build_apps”, a opcje, które jej przekazujemy, opisują, jak zbudować naszą aplikację. Obejmuje to takie rzeczy, jak platformy, dla których chcemy budować, które pliki lub typy plików chcemy uwzględnić lub wykluczyć z produktu końcowego i tak dalej.

Zatem:

```python
from setuptools import setup

setup(
    name="Amiga Boing Ball",
    options={
        "build_apps": {
            # Tutaj znajdują się opcje kompilacji...
        }
    }
)
```

Co pozostawia do wypełnienia tylko opcje kompilacji.

Użyjemy sześciu: „include_patterns”, „gui_apps”, „plugins”, „platforms”, „log_filename” i „log_append”.
* „include_patterns” wskazuje, które pliki powinny być częścią produktu końcowego. Niektóre z nich zostaną przetworzone (np. pliki „egg” zostaną przekonwertowane na pliki „bam”), podczas gdy inne zostaną dołączone bez zmian.
  * Mogą to być określone nazwy plików lub wzorce plików.
  * Jeśli mamy pliki, które chcemy wykluczyć, możemy to zrobić za pomocą „exclude_patterns”.
* „gui_apps” wskazuje, że budujemy aplikację, która otwiera okno, a także wskazuje lokalizację „głównego” pliku Pythona aplikacji. Gdybyśmy chcieli uruchomić z konsoli, moglibyśmy zamiast tego użyć „console_apps”. Zauważcie, że gdybyśmy chcieli, moglibyśmy zbudować wiele aplikacji, zarówno gui, jak i konsolę.

* „plugins” wskazują, których wtyczek Panda3D chcemy użyć. Obejmuje to takie rzeczy jak OpenGL, OpenAL, FMod, obsługę dodatkowych typów plików modeli i tak dalej.
* „platforms” to po prostu systemy operacyjne, dla których chcemy budować (aw niektórych przypadkach budować dla systemów 32- lub 64-bitowych). Uwzględnię opcje Windows, Mac i Linux w sekcji „platforms” poniżej - po prostu usuńcie te, których nie chcecie.

* "log_filename" mówi Panda3D, jak nazwać plik dziennika, który tworzy w celu raportowania różnych danych wyjściowych - w szczególności takich jak ostrzeżenia i błędy.
  * To może wydawać się nieistotne – w końcu do tej pory byliśmy w stanie zobaczyć te dane wyjściowe w naszej konsoli! Jednak w systemach Windows i Mac kompilacje dystrybuowalne nie dadzą takich danych wyjściowych poza plikiem dziennika. To może sprawić, że debugowanie wszelkich problemów, które pojawią się w takiej kompilacji, będzie naprawdę bardzo trudne! Dlatego bardzo dobrym pomysłem jest posiadanie dziennika!
* „log_append” po prostu wskazuje, czy Panda3D powinna zezwolić plikowi dziennika na gromadzenie danych wyjściowych z wielu przebiegów, czy też wyczyścić go i rozpocząć od nowa przy każdym przebiegu.

W ten sposób kończymy na tym:

```python
from setuptools import setup

setup(
    name="Amiga Boing Ball",
    options={
        "build_apps": {
            # Pliki, które chcemy dołączyć. Konkretnie:
            # * Wszystkie nasze pliki graficzne (.png)
            # * Wszystkie nasze pliki dźwiękowe i muzyczne
            #   (.ogg)
            # * Wszystkie nasze pliki tekstowe (.txt)
            # * Wszystkie nasze modele 3D (.egg)
            #    - Zostaną one automatycznie prze-konwertowane
            #      na pliki .bam
            # * Wszystkie nasze modele 3D (.bam)
            # * I wszystkie nasze pliki czcionek
            #   (w folderze „Font”)
            "include_patterns": [
                "**/*.png",
                "**/*.ogg",
                "**/*.txt",
                "**/*.egg",
                "**/*.bam",
                "Fonts/*"
            ],
            # Chcemy mieć aplikację gui, a nasz "główny" plik
            # Pythona to "2a_amiga.py"
            "gui_apps": {
                "Amiga Boing Ball": "2a_amiga.py"
            },
            # Wtyczki, których używamy. W szczególności
            # używamy OpenGL i dźwięku OpenAL
            "plugins": [
                "pandagl",
                "p3openal_audio"
            ],
            # Platformy, dla których budujemy.
            # Usuńcie te, których nie chcecie.
            "platforms": [
                "manylinux2010_x86_64",
                "macosx_10_9_x86_64",
                "win_amd64"
            ],
            # Nazwa naszego pliku dziennika. Zachowujemy
            # krótką nazwę katalogu — nasz tytuł jest dość
            # długi — i umieszczamy plik w katalogu
            # „app-data” użytkownika.
            "log_filename":
                "$USER_APPDATA/AmigaBoingBall/output.log",
            # Zamiast zezwalać na gromadzenie danych dziennika,
            # decydujemy się na rozpoczęcie dziennika od nowa
            # przy każdym uruchomieniu.
            "log_append": False
        }
    }
)
```

Zauważcie, że jeśli całkowicie pominiemy sekcję „platforms”, system budowania automatycznie zbuduje się dla wszystkich domyślnych platform. Domyślne są trzy, które zawarliśmy w naszym "setup.py" powyżej.

Zauważcie również, że konkretne nazwy różnych platform będą mogły się zmienić. Lepiej sprawdzić aktualne nazwy platform, z których zamierzacie korzystać!

W skrypcie „setup.py” dostępnych jest wiele innych opcji. Pełniejszą listę i bardziej szczegółowe wyjaśnienie znajdziecie w instrukcji!

Tak więc, mając to wszystko na swoim miejscu, pozostaje tylko jedna rzecz do zrobienia: faktycznie uruchomić polecenie, które ustawia naszą aplikację!

Aby to zrobić, otwieramy konsolę i uruchamiamy nasz skrypt „setup.py”, przekazując do niego polecenie „bdist_apps”. Spowoduje to, że system budowania najpierw zbuduje naszą dystrybucyjną aplikację, a następnie spakuje ją w ładnie przenośnym formacie. (Aktualnie wszystkie formaty opakowań są archiwami).

Polecenie wygląda następująco:

```sh
python3 setup.py bdist_apps
```